In [1]:
import pandas as pd
import json
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn import linear_model
from datetime import datetime
from scipy.stats import skew
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectFromModel
userdatapath = '/home/titli/Documents/Springboard1/test.csv'
selecteddatapath = '/home/titli/Documents/Springboard1/test1.csv'
reviewfile= '/home/titli/Documents/Springboard1/dataset/review.json'
businessfile = '/home/titli/Documents/Springboard1/dataset/business.json'
numsamples=1000
rowsamples=200
pd.options.display.max_columns = 200

In [2]:
old=pd.read_csv(userdatapath, header=0,  index_col=0, nrows=numsamples)

#100 points

selectedrows=[0,8,12,15,19,25,28,29,45,47,53,54,65,71,85,87,94,101,106,114,126,127,174,175, \
              176,177,178,193,197,206,209,210,222,224,227,244,248,250,255,258, \
              275,276,279,284,289,290,303,309,313,326,331,332,338,349,356,364,366,381,387,390,392,397,\
              404,408,418,444,445,455,470,474,480,482,489,495,497,500,507,517,520,529, \
              1,2,3,4,5,6,7,9,10,13,17,18,21,24,26,27,30,31,32,33,34,35, \
              183,184,185,186,187,190,194,196,199,201,205,208,232,235,240,249,266,269,\
              327,328,329,333,334,335, 336,337,339,340,341,342,343,345,346,347,350,351,352,353,359,360,362,\
              363,365,367,368,369,627,374,376,380,382,383,385,386,388, \
              128,136,142,149,153,155,167,268,274,555,569,582,583,591,597,601,611,614,622,631, \
              36,37,38,39,40,42,43,44,270,273,389,391,393,394,395,396,400,471,473,479,623,624,626]                         
              
              
#fake=174,175,176,177,178,193,197,206,209,210,222,224,227,244,248,250,255,258,268,274,275,276,279,284,289,290,303, \
#309,313,326
#183,184,185,186,187,190,194,196,199,201,205,208,232,235,240,249,266,269,270,273,278,281,282,283,285,286,287,291,292,293,\
#295,296,324,325
inter=[]
for row in selectedrows:
    line= old.iloc[row]
    inter.append(line)
old_1=pd.DataFrame(inter)
old_1.set_index('user_id')
old_1.head(10)
old_1.to_csv(selecteddatapath)
small= pd.read_csv(selecteddatapath, header=0)
cols = 'average_stars 	review_count compliment_cool 	compliment_cute 	compliment_funny 	 \
compliment_hot 	compliment_list 	compliment_more 	compliment_note 	compliment_photos 	 \
compliment_plain 	cool  	fans funny '.split()
old_2=small[cols]
old_2.head()

,average_stars,review_count,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,cool,fans,funny
0,4.67,8,0,0,0,0,0,0,0,0,1,0,0,0
1,1.00,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1.67,2,0,0,0,0,0,0,0,0,0,0,0,0
3,3.00,2,0,0,0,0,0,0,0,0,0,1,0,0
4,3.73,48,1,0,1,2,0,0,1,0,1,2,3,6


In [3]:
from pandas.io.json import json_normalize
def getreviews(user_ids):
    """ gets one text review for each user in user_ids list"""
    result=[]
    with open(reviewfile) as f:
         for line in f:
             record = json.loads(line)
             if record['user_id'] in user_ids: 
                result.append(record)
             if len(result) >= 1000:  #numsamples
                break
    reviewdata = pd.DataFrame(result)
    return reviewdata

In [4]:
user_ids=[]
for counter, value in enumerate(selectedrows):
    user_ids.append(old['user_id'].iloc[value])
review_texts_all = getreviews(user_ids)

In [5]:
review_texts_all.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,K1tLIHP5DLZG5NYIt83iYA,0,2017-10-02,0,7AIRjMM5y80d8WlcyY32uQ,5,Ken and Debra are amazing real estate agents! ...,0,RJaOM110aQlyfab_mYBw4w
1,PmgbmK6wG0S-Nndo1becBA,0,2016-04-23,0,z8OYnWR_s62vtHp5GCmEWQ,1,Absolute worst service. I have never been trea...,1,akFg3cvYqbihkdYUIjTauQ
2,rL-hprXkcvfYSL9kWBFglQ,0,2014-02-23,0,JCUtZ8QvWDdjSEm7Km9W-w,5,I will be back. Unlike the old Native New Yor...,0,ddhNJ-nbjwjHoSth6qXJ8g
3,p_22wrx3GmAzlbrGA6LyYg,0,2014-03-10,0,-nj0swfsRUk83zgpWibT0A,5,Proud to support family owned restaurant. I l...,0,ddhNJ-nbjwjHoSth6qXJ8g
4,_ZfjpSEO5ntk-1hbnwCR4g,0,2015-10-10,1,fVL9vfsWydvzxVfmEiERtg,2,Stayed here for a few nights on a random getaw...,0,gb8PYSCKlvPbMWnqIWXmlQ


In [6]:
df1=[]
for i in range(rowsamples):
    data = review_texts_all[review_texts_all.user_id == user_ids[i]]
    n = review_texts_all[review_texts_all.user_id == user_ids[i]].count()
    if len(n) > 1:
       data1 = data.iloc[0]
       df1.append(data1)

review_texts = pd.DataFrame(df1)

In [7]:
df1 = []
for k in range(rowsamples):
    df1.append({'rev_length': len(review_texts['text'].iloc[k]), 'rev_use': review_texts['useful'].iloc[k],\
               'rev_stars': review_texts['stars'].iloc[k], 'rev_date': review_texts['date'].iloc[k], \
               'rev_buss': review_texts['business_id'].iloc[k]})
revlen=pd.DataFrame(df1)

In [8]:
df1 = []
for k in range(rowsamples):
    
    label=0
    Xlen=old_1['review_count'].iloc[k]
    
    if Xlen<2:
       label=0
    elif np.logical_and((Xlen>=2), (Xlen<=20)):
       label=1
    elif np.logical_and((Xlen>20), (Xlen<=50)):
       label=2
    elif np.logical_and((Xlen>50), (Xlen<=100)):
       label=3
    else:
      label=4
    df1.append({'rev_count_label': label})    
Xlen_label=pd.DataFrame(df1)

In [9]:
df2 = []
for k in range(0,rowsamples):
    count=0
    label=0
    Xfrnd=(old_1['friends'].iloc[[k]].values).tolist()
    dfrnd = (", ".join(Xfrnd)).split(',')[1:-1]
    if dfrnd is None:
       break
    else:
       count=len(dfrnd)
       if count==0:
          label=0
       elif np.logical_and((count>0), (count<=50)):
          label=1
       elif np.logical_and((count>50), (count<=100)):
          label=2
          
       else:
          label=3
       df2.append({'friend_count': count,'friend_label': label})    
frnd=pd.DataFrame(df2)

In [10]:
from pandas.io.json import json_normalize
def getbusiness(business_ids):
    """ gets stars review for each business in business_ids list"""
    result=[]
    with open(businessfile) as f:
         for line in f:
             record = json.loads(line)
             if record['business_id'] in business_ids: 
                result.append(record)
             if len(result) >= numsamples:
                break
    businessdata = pd.DataFrame(result)
    return businessdata

In [11]:
business_ids = revlen['rev_buss'].iloc[:numsamples].values.tolist()
businessstars = pd.DataFrame({'buss_star': getbusiness(business_ids)['stars'], 'buss_review': getbusiness(business_ids)['review_count']  })

In [12]:
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
english_stops = set(stopwords.words('english'))
def alphalength(text):
# Import WordNetLemmatizer
    tokens = word_tokenize(text)
    lower_tokens = [t.lower() for t in tokens]
# Retain alphabetic words: alpha_only
    alpha_only = [t for t in lower_tokens if t.isalpha()]

    length_alpha = len(alpha_only)

# Print the 10 most common tokens
    return(length_alpha)

In [13]:
df1=[]
for i in range(rowsamples):
    a1=alphalength(review_texts['text'].iloc[i])
    df1.append(a1)
Alphalength=pd.DataFrame({'alpha_length': df1})
Alphalength.head()


,alpha_length
0,98
1,139
2,46
3,35
4,279


In [14]:
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
english_stops = set(stopwords.words('english'))
def wordstop(text):
# Import WordNetLemmatizer
    tokens = word_tokenize(text)
    lower_tokens = [t.lower() for t in tokens]
# Retain alphabetic words: alpha_only
    alpha_only = [t for t in lower_tokens if t.isalpha()]

       
    # Remove all stop words: no_stops
    no_stops = [t for t in alpha_only if t not in english_stops]

    length_alpha = len(no_stops)
    
# Print the 10 most common tokens
    return(length_alpha)

In [15]:
df1=[]
for i in range(rowsamples):
    a1=wordstop(review_texts['text'].iloc[i])
    df1.append(a1)
stopwords=pd.DataFrame({'nostop_length': df1})
stopwords.head()


,nostop_length
0,46
1,72
2,18
3,18
4,137


In [16]:
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
english_stops = set(stopwords.words('english'))
def wordlemmatize(text):
# Import WordNetLemmatizer
    tokens = word_tokenize(text)
    lower_tokens = [t.lower() for t in tokens]
# Retain alphabetic words: alpha_only
    alpha_only = [t for t in lower_tokens if t.isalpha()]

       
    # Remove all stop words: no_stops
    no_stops = [t for t in alpha_only if t not in english_stops]
    
    #  Instantiate the WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    # Lemmatize all tokens into a new list: lemmatized
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
    
    length_lemmatize = len(lemmatized)
    
# Print the 10 most common tokens
    return(length_lemmatize)

In [17]:
df1=[]
for i in range(rowsamples):
    a1=wordlemmatize(review_texts['text'].iloc[i])
    df1.append(a1)
lemmatizewords=pd.DataFrame({'length_lemmatize': df1})
lemmatizewords.head()

,length_lemmatize
0,46
1,72
2,18
3,18
4,137


In [18]:
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
english_stops = set(stopwords.words('english'))
def wordpos(text):
# Import WordNetLemmatizer
    tokens = word_tokenize(text)
    
    lower_tokens = [t.lower() for t in tokens]
    
    counts = Counter([j for i,j in pos_tag(lower_tokens)])
    

    return (counts)

In [19]:
df1=[]
for i in range(rowsamples):
    a1=wordpos(review_texts['text'].iloc[i])
    df1.append(a1)

Netpos=pd.DataFrame(df1)
Netpos.head()
POS_columns = 'NN 	NNP 	NNS 	PDT 	POS 	\
PRP 	PRP$ 	RB 	RBR 	RBS 	RP 		VB 	VBD 	VBG 	VBN 	\
VBP 	VBZ 	WDT 	WP 	WRB'.split()
postotal=Netpos[POS_columns]
postotal.fillna(value= 0.0, inplace=True)
postotal.head()

/home/titli/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,NN,NNP,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WRB
0,22,0.0,3.0,0.0,1.0,8.0,3.0,8.0,0.0,0.0,0.0,5.0,5.0,0.0,4.0,1.0,3.0,1.0,0.0,1.0
1,30,0.0,8.0,0.0,0.0,5.0,9.0,9.0,0.0,0.0,0.0,7.0,11.0,1.0,4.0,3.0,1.0,1.0,0.0,1.0
2,10,0.0,3.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,2.0,6.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0
3,6,0.0,3.0,0.0,0.0,4.0,1.0,4.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,55,0.0,9.0,0.0,0.0,13.0,4.0,17.0,1.0,0.0,0.0,14.0,15.0,3.0,10.0,6.0,16.0,1.0,2.0,1.0


In [20]:
d={'Faker':[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,\
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,\
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}
faker= pd.DataFrame(data=d)
new = pd.concat([old_2,  revlen['rev_length'] , revlen['rev_stars'], revlen['rev_use'], frnd, Xlen_label, \
                 businessstars, Alphalength, lemmatizewords, postotal, faker], axis=1)
#
new['buss_star'].replace(to_replace= 'NaN', value=np.nan, inplace=True)
new['buss_star'].fillna(value= 1, inplace=True)  # business not found 
new['buss_review'].replace(to_replace= 'NaN', value=np.nan, inplace=True)
new['buss_review'].fillna(value= 0, inplace=True)  # business not found 
cleaned_data=new
dataset1=cleaned_data[cleaned_data['Faker']==1]
dataset0=cleaned_data[cleaned_data['Faker']==0]

cleaned_data.describe()



,average_stars,review_count,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,cool,fans,funny,rev_length,rev_stars,rev_use,friend_count,friend_label,rev_count_label,buss_star,buss_review,alpha_length,length_lemmatize,NN,NNP,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WRB,Faker
count,200.000000,200.000000,200.000000,200.0,200.000000,200.000000,200.0,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,3.688800,8.280000,0.170000,0.0,0.170000,0.080000,0.0,0.035000,0.150000,0.080000,0.250000,0.385000,0.630000,0.750000,534.855000,3.645000,0.705000,13.735000,0.195000,0.855000,3.762500,347.730000,98.620000,47.720000,19.630000,0.020000,3.935000,0.090000,0.180000,5.520000,2.420000,8.150000,0.160000,0.050000,0.53000,5.015000,7.220000,1.735000,1.945000,2.680000,2.035000,0.385000,0.360000,0.525000,0.500000
std,1.253741,16.662225,0.913935,0.0,0.913935,0.452353,0.0,0.306995,0.787784,0.752617,0.944436,1.558499,5.567593,4.385621,536.602446,1.662164,1.424119,58.262832,0.685071,0.660421,0.896584,769.966779,101.510975,46.028302,18.610612,0.172478,4.260462,0.303911,0.537629,8.171503,2.887088,9.122147,0.442015,0.218492,1.10235,5.965123,8.557321,2.462742,2.429297,3.990894,3.295158,0.965107,0.820834,1.036701,0.501255
min,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,9.000000,7.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,216.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.500000,26.250000,39.000000,20.000000,8.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,3.000000,0.000000,0.000000,0.00000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.915000,3.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,389.000000,5.000000,0.000000,0.000000,0.000000,1.000000,4.000000,90.500000,72.500000,34.500000,14.000000,0.000000,3.000000,0.000000,0.000000,3.000000,1.000000,6.000000,0.000000,0.000000,0.00000,4.000000,5.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.500000
75%,5.000000,7.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,620.250000,5.000000,1.000000,0.000000,0.000000,1.000000,4.500000,232.250000,113.000000,55.250000,22.250000,0.000000,5.000000,0.000000,0.000000,6.000000,3.000000,10.000000,0.000000,0.000000,1.00000,6.000000,9.000000,2.000000,3.000000,3.000000,3.000000,0.000000,0.000000,1.000000,1.000000
max,5.000000,154.000000,8.000000,0.0,8.000000,4.000000,0.0,4.000000,9.000000,10.000000,7.000000,15.000000,77.000000,56.000000,4950.000000,5.000000,10.000000,428.000000,3.000000,4.000000,5.000000,5950.000000,926.000000,407.000000,150.000000,2.000000,32.000000,2.000000,4.000000,77.000000,15.000000,93.000000,3.000000,1.000000,8.00000,52.000000,47.000000,15.000000,13.000000,43.000000,38.000000,9.000000,4.000000,8.000000,1.000000


In [21]:
cleaned_data.to_csv('cleaned_data.csv')